In [ ]:
!pip install PyWavelets

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import ccf
from scipy.stats import entropy
from google.colab import drive
from datetime import datetime
import pandas as pd
import numpy as np
import pywt

from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu
from scipy.stats import ks_2samp

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading Daily Data

In [ ]:
def remove_index(df):
  df.index = df['Date']
  df.drop(columns=['Date'], inplace=True)

In [ ]:
def transfer_data(data, df):
  for column in df.columns:
    data[column] = 0
    data[column].update(df[column])

In [ ]:
filename = 'daily_garch_data.csv'
garch_data = pd.read_csv(path + filename)

In [ ]:
remove_index(garch_data)

In [ ]:
garch_data

,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)","('GARCH', 'studentst', 0)","('GARCH', 'studentst', 1)","('GARCH', 'skewstudent', 0)","('GARCH', 'skewstudent', 1)","('GARCH', 'skewt', 0)","('GARCH', 'skewt', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)"
Date,,,,,,,,,,,,
2000-02-02,0.015777,0.014045,0.015777,0.014045,0.015643,0.015741,0.014138,0.014088,0.014138,0.014088,0.014461,0.015741
2000-02-03,0.014762,0.013195,0.014762,0.013195,0.014919,0.013317,0.015187,0.013105,0.015187,0.013105,0.015200,0.013259
2000-02-04,0.013612,0.013193,0.013612,0.013193,0.013797,0.013305,0.013541,0.013106,0.013541,0.013106,0.013906,0.013335
2000-02-07,0.013027,0.012561,0.013027,0.012561,0.012884,0.012621,0.012062,0.011851,0.012062,0.011851,0.012833,0.012574
2000-02-08,0.012541,0.011799,0.012541,0.011799,0.012734,0.011860,0.480832,0.011318,0.480832,0.011318,0.012765,0.011856
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.010817,0.010322,0.010817,0.010322,0.010210,0.018468,0.015242,0.023673,0.015242,0.023673,0.009141,0.009129
2024-08-27,0.009021,0.006857,0.009021,0.006857,0.032060,0.008563,0.008825,0.009353,0.008825,0.009353,0.009020,0.008028
2024-08-28,0.009278,0.007415,0.009278,0.007415,0.009284,0.011995,0.009134,0.017352,0.009134,0.017352,0.009192,0.007158


In [ ]:
filename = 'returns_&_volume.csv'
returns_volume = pd.read_csv(path + filename)

In [ ]:
remove_index(returns_volume)

In [ ]:
returns_volume.index = garch_data.index

In [ ]:
filename = 'daily_egarch_data.csv'
egarch_data = pd.read_csv(path + filename)

In [ ]:
remove_index(egarch_data)

In [ ]:
filename = 'daily_figarch_data.csv'
figarch_data = pd.read_csv(path + filename)

In [ ]:
remove_index(figarch_data)

In [ ]:
filename = 'daily_aparch_data.csv'
aparch_data = pd.read_csv(path + filename)

In [ ]:
remove_index(aparch_data)

In [ ]:
filename = 'daily_harch_data.csv'
harch_data = pd.read_csv(path + filename)

In [ ]:
remove_index(harch_data)

In [ ]:
data = pd.concat([returns_volume, garch_data, egarch_data, figarch_data, aparch_data, harch_data], axis=1)

In [ ]:
filename = 'daily_volatility_proxies.csv'
volatility_proxies = pd.read_csv(path + filename)

In [ ]:
remove_index(volatility_proxies)

In [ ]:
volatility_proxies.index = garch_data.index

In [ ]:
filename = 'extra_daily_data.csv'
extra_data = pd.read_csv(path + filename)

In [ ]:
# extra_data['Date'] = extra_data['Unnamed: 0']
extra_data.index = extra_data['Date']
extra_data.drop(columns=['Date'], inplace=True)

In [ ]:
transfer_data(data, extra_data)

<ipython-input-115-12b073d0f83e>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].update(df[column])
<ipython-input-115-12b073d0f83e>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[32.04999924 32.04999924 32.04999924 ... 74.51999664 75.91000366
 73.55000305]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[column].update(df[column])
<ipython-input-115-12b073d0f83e>:4: FutureWarning: A

In [ ]:
filename = 'gt_daily_data.csv'
gt_data = pd.read_csv(path + filename)

In [ ]:
gt_data['Date'] = gt_data['Unnamed: 0']
gt_data.index = gt_data['Date']
gt_data.drop(columns=['Date', 'Unnamed: 0'], inplace=True)

In [ ]:
transfer_data(data, gt_data)

<ipython-input-115-12b073d0f83e>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].update(df[column])
<ipython-input-115-12b073d0f83e>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[74.         74.         74.         ... 53.42857143 53.57142857
 53.71428571]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[column].update(df[column])


In [ ]:
data = pd.concat([data, volatility_proxies], axis=1)

In [ ]:
data.columns

Index(['log_returns', 'volume', '('GARCH', 'normal', 0)',
       '('GARCH', 'normal', 1)', '('GARCH', 'gaussian', 0)',
       '('GARCH', 'gaussian', 1)', '('GARCH', 'studentst', 0)',
       '('GARCH', 'studentst', 1)', '('GARCH', 'skewstudent', 0)',
       '('GARCH', 'skewstudent', 1)', '('GARCH', 'skewt', 0)',
       '('GARCH', 'skewt', 1)', '('GARCH', 'ged', 0)', '('GARCH', 'ged', 1)',
       '('EGARCH', 'normal', 0)', '('EGARCH', 'normal', 1)',
       '('EGARCH', 'gaussian', 0)', '('EGARCH', 'gaussian', 1)',
       '('EGARCH', 'studentst', 0)', '('EGARCH', 'studentst', 1)',
       '('EGARCH', 'skewstudent', 0)', '('EGARCH', 'skewstudent', 1)',
       '('EGARCH', 'skewt', 0)', '('EGARCH', 'skewt', 1)',
       '('EGARCH', 'ged', 0)', '('EGARCH', 'ged', 1)',
       '('FIGARCH', 'normal', 0)', '('FIGARCH', 'normal', 1)',
       '('FIGARCH', 'gaussian', 0)', '('FIGARCH', 'gaussian', 1)',
       '('FIGARCH', 'studentst', 0)', '('FIGARCH', 'studentst', 1)',
       '('FIGARCH', 'skewstudent

# Data Normalization

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [ ]:
scaled_df = pd.DataFrame(scaled_features, columns=data.columns, index=data.index)

# Wavelet transformation

In [ ]:
def best_tranformation(transformed_data, original_data):
  # Calculate statistics, reconstruction errors, and perform tests
  statistics, reconstruction_errors = calculate_statistics_and_errors(transformed_data, original_data)
  t_test_results = perform_ttest(original_data, transformed_data)
  anova_results = perform_anova(original_data, transformed_data)
  chi2_results = perform_chi2_test(original_data, transformed_data)
  mannwhitney_results = perform_mannwhitney_test(original_data, transformed_data)
  ks_test_results = perform_ks_test(original_data, transformed_data)

  # Combine reconstruction errors and test results into a single score
  mean_reconstruction_error = np.mean(list(reconstruction_errors.values()))
  test_scores = np.mean([result['p_val'] for result in t_test_results.values()] +
                        [result['p_val'] for result in anova_results.values()] +
                        [result['p_val'] for result in chi2_results.values()] +
                        [result['p_val'] for result in mannwhitney_results.values()] +
                        [result['p_val'] for result in ks_test_results.values()])

  total_score = mean_reconstruction_error + (1 - test_scores)  # Combine scores (lower is better)

  return total_score

In [ ]:
# def apply_continuous_wavelet_transform(original_data, wavelet, scale, method):
#     coefficients, frequencies = pywt.cwt(original_data, scales=scale, wavelet=wavelet)

#     if method == 'energy':
#         result = np.sum(coefficients ** 2, axis=0)
#     elif method == 'mean':
#         result = np.mean(coefficients, axis=0)
#     elif method == 'variance':
#         result = np.var(coefficients, axis=0)
#     else:
#         raise ValueError(f"Method {method} is not recognized")

#     return result

In [ ]:
def apply_continuous_wavelet_transform(data, wavelet, scale):
  transformed_df = pd.DataFrame(index=data.index)

  for column in data.columns:
    original_data = data[column].values

    # Apply continuous wavelet transform (CWT)
    coefficients, frequencies = pywt.cwt(original_data, scale, wavelet)

    # Calculate wavelet energy for each scale
    wavelet_energy = np.sum(coefficients ** 2, axis=0)

    # Store the wavelet energy in the DataFrame
    transformed_df[column] = wavelet_energy

  return transformed_df

In [ ]:
def apply_discrete_wavelet_transform(data, wavelet, max_level):
  transformed_df = pd.DataFrame(index=data.index)

  for column in data.columns:
    original_data = data[column].values

    # Apply discrete wavelet transform (DWT)
    coeffs = pywt.wavedec(original_data, wavelet, level=max_level)

    # Concatenate coefficients for multi-resolution features
    multi_res_features = np.concatenate(coeffs)

    # Truncate or pad features to match the length of the original data
    if len(multi_res_features) > len(original_data):
      multi_res_features = multi_res_features[:len(original_data)]
    elif len(multi_res_features) < len(original_data):
      # Padding with NaNs if necessary
      multi_res_features = np.pad(multi_res_features, (0, len(original_data) - len(multi_res_features)), mode='constant', constant_values=np.nan)

    # Store the multi-resolution features in the DataFrame
    transformed_df[column] = multi_res_features

  return transformed_df

In [ ]:
# def apply_discrete_wavelet_transform(original_data, wavelet, level, method):
#     coefficients = pywt.wavedec(original_data, wavelet, level=level)

#     if method == 'energy':
#         result = np.sum([np.sum(c ** 2) for c in coefficients])
#     elif method == 'mean':
#         result = np.mean([np.mean(c) for c in coefficients])
#     elif method == 'variance':
#         result = np.var([np.var(c) for c in coefficients])
#     else:
#         raise ValueError(f"Method {method} is not recognized")

#     return result

In [ ]:
# def evaluate_wavelet_combinations(original_data, wavelet_transform_fn, wavelets, scales_or_levels, wavelet_type, methods):
#     best_combination = None
#     best_method = None
#     best_score = float('inf')  # Minimum score corresponds to best performance

#     # Store results for comparison
#     best_statistics = None
#     best_reconstruction_errors = None
#     best_test_results = None

#     for wavelet in wavelets:
#         for scale_or_level in scales_or_levels:
#             for method in methods:
#                 # Apply the chosen wavelet transform (CWT or DWT)
#                 if wavelet_type == 'continuous':
#                     transformed_data = wavelet_transform_fn(original_data, wavelet, scale_or_level, method)
#                 else:
#                     transformed_data = wavelet_transform_fn(original_data, wavelet, scale_or_level, method)

#                 # Calculate statistics, reconstruction errors, and perform tests
#                 statistics, reconstruction_errors = calculate_statistics_and_errors(transformed_data, original_data)
#                 t_test_results = perform_ttest(original_data, transformed_data)
#                 anova_results = perform_anova(original_data, transformed_data)
#                 chi2_results = perform_chi2_test(original_data, transformed_data)
#                 mannwhitney_results = perform_mannwhitney_test(original_data, transformed_data)
#                 ks_test_results = perform_ks_test(original_data, transformed_data)

#                 # Combine reconstruction errors and test results into a single score
#                 mean_reconstruction_error = np.mean(list(reconstruction_errors.values()))
#                 test_scores = np.mean([result['p_val'] for result in t_test_results.values()] +
#                                       [result['p_val'] for result in anova_results.values()] +
#                                       [result['p_val'] for result in chi2_results.values()] +
#                                       [result['p_val'] for result in mannwhitney_results.values()] +
#                                       [result['p_val'] for result in ks_test_results.values()])

#                 total_score = mean_reconstruction_error + (1 - test_scores)  # Combine scores (lower is better)

#                 # If this combination is better, update best combination and best method
#                 if total_score < best_score:
#                     best_score = total_score
#                     best_combination = (wavelet, scale_or_level)
#                     best_method = method
#                     best_statistics = statistics
#                     best_reconstruction_errors = reconstruction_errors
#                     best_test_results = {
#                         't_test': t_test_results,
#                         'anova': anova_results,
#                         'chi2': chi2_results,
#                         'mannwhitney': mannwhitney_results,
#                         'ks_test': ks_test_results
#                     }

#     return best_combination, best_method, best_statistics, best_reconstruction_errors, best_test_results

In [ ]:
# def calculate_statistics_and_errors(transformed_data, original_data):
#     statistics = {}
#     reconstruction_errors = {}

#     # Calculate mean and standard deviation for each column
#     for column in original_data.columns:
#         original_mean = np.mean(original_data[column])
#         original_std = np.std(original_data[column])
#         transformed_mean = np.mean(transformed_data[column])
#         transformed_std = np.std(transformed_data[column])

#         statistics[column] = {
#             'original_mean': original_mean,
#             'original_std': original_std,
#             'transformed_mean': transformed_mean,
#             'transformed_std': transformed_std
#         }

#         # Calculate reconstruction error (mean squared error)
#         reconstruction_errors[column] = np.mean((original_data[column] - transformed_data[column]) ** 2)

#     return statistics, reconstruction_errors

In [ ]:
# def perform_ttest(original_data, transformed_data):
#     t_test_results = {}

#     # Perform t-test for each column
#     for column in original_data.columns:
#         t_stat, p_val = ttest_ind(original_data[column], transformed_data[column])

#         t_test_results[column] = {
#             't_stat': t_stat,
#             'p_val': p_val
#         }

#     return t_test_results

In [ ]:
# def perform_anova(original_data, transformed_data):
#     anova_results = {}


#     # Perform ANOVA for each column
#     for column in original_data.columns:
#         f_stat, p_val = f_oneway(np.abs(original_data[column]), np.abs(transformed_data[column]))

#         anova_results[column] = {
#             'f_stat': f_stat,
#             'p_val': p_val
#         }

#     return anova_results

In [ ]:
# def perform_chi2_test(original_data, transformed_data):
#     chi2_results = {}

#     # Perform Chi-Square test for each column (only works with categorical/discrete data)
#     for column in original_data.columns:
#         contingency_table = pd.crosstab(original_data[column], transformed_data[column])
#         chi2_stat, p_val, _, _ = chi2_contingency(contingency_table)

#         chi2_results[column] = {
#             'chi2_stat': chi2_stat,
#             'p_val': p_val
#         }

#     return chi2_results

In [ ]:
def perform_mannwhitney_test(original_data, transformed_data):
    mannwhitney_results = {}

    # Perform Mann-Whitney U test for each column
    for column in original_data.columns:
        u_stat, p_val = mannwhitneyu(original_data[column], transformed_data[column])

        mannwhitney_results[column] = {
            'u_stat': u_stat,
            'p_val': p_val
        }

    return mannwhitney_results

In [ ]:
def perform_ks_test(original_data, transformed_data):
    ks_test_results = {}

    # Perform KS test for each column
    for column in original_data.columns:
        ks_stat, p_val = ks_2samp(original_data[column], transformed_data[column])

        ks_test_results[column] = {
            'ks_stat': ks_stat,
            'p_val': p_val
        }

    return ks_test_results

In [ ]:
# Define the input parameters
methods = ['energy', 'mean', 'variance']
continuous_wavelets = ['cmor', 'morl', 'mexh']
discrete_wavelets = ['haar', 'db4', 'db6', 'db8']
scales = [np.arange(1, 42), np.arange(1, 20), np.arange(1, 30), np.arange(1, 50),
          np.arange(1, 60), np.arange(10, 50), np.arange(20, 60), np.arange(1, 128)]
max_level = [2, 3, 4]

In [ ]:
# # Evaluate for Continuous Wavelet Transform
# best_cwt_combination, best_cwt_method, best_cwt_stats, best_cwt_errors, best_cwt_tests = evaluate_wavelet_combinations(
#     scaled_df, apply_continuous_wavelet_transform, continuous_wavelets, scales, 'continuous', methods
# )

In [ ]:
# # Evaluate for Discrete Wavelet Transform
# best_dwt_combination, best_dwt_method, best_dwt_stats, best_dwt_errors, best_dwt_tests = evaluate_wavelet_combinations(
#     scaled_df, apply_discrete_wavelet_transform, discrete_wavelets, max_level, 'discrete', methods
# )

In [ ]:
# # Print the continuous results
# print(f"Best CWT Combination: {best_cwt_combination}")
# print("Best CWT Statistics:", best_cwt_stats)
# print("Best CWT Reconstruction Errors:", best_cwt_errors)
# print("Best CWT Test Results:", best_cwt_tests)
# print(f"Best Calculation Method: {best_cwt_method}")

In [ ]:
# # Print the discrete results
# print(f"Best DWT Combination: {best_dwt_combination}")
# print("Best DWT Statistics:", best_dwt_stats)
# print("Best DWT Reconstruction Errors:", best_dwt_errors)
# print("Best DWT Test Results:", best_dwt_tests)
# print(f"Best Calculation Method: {best_dwt_method}")

In [ ]:
ndwt_daily_data = apply_discrete_wavelet_transform(scaled_df, 'haar', 3)

In [ ]:
ncwt_daily_data = apply_continuous_wavelet_transform(scaled_df, 'cmor', np.arange(1, 20))

/usr/local/lib/python3.10/dist-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow the format cmorB-C, where B and C are floats representing the bandwidth frequency and center frequency, respectively (example, for backward compatibility: cmor = cmor1.0-0.5).
  wavelet = DiscreteContinuousWavelet(wavelet)
/usr/local/lib/python3.10/dist-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow the format cmorB-C, where B and C are floats representing the bandwidth frequency and center frequency, respectively (example, for backward compatibility: cmor = cmor1.0-0.5).
  wavelet = DiscreteContinuousWavelet(wavelet)
/usr/local/lib/python3.10/dist-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow th

In [ ]:
dwt_daily_data = apply_discrete_wavelet_transform(data, 'haar', 3)

In [ ]:
cwt_daily_data = apply_continuous_wavelet_transform(data, 'cmor', np.arange(1, 20))

/usr/local/lib/python3.10/dist-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow the format cmorB-C, where B and C are floats representing the bandwidth frequency and center frequency, respectively (example, for backward compatibility: cmor = cmor1.0-0.5).
  wavelet = DiscreteContinuousWavelet(wavelet)
/usr/local/lib/python3.10/dist-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow the format cmorB-C, where B and C are floats representing the bandwidth frequency and center frequency, respectively (example, for backward compatibility: cmor = cmor1.0-0.5).
  wavelet = DiscreteContinuousWavelet(wavelet)
/usr/local/lib/python3.10/dist-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow th

In [ ]:
# Option 1: Use real part only
real_cwt_daily_data = cwt_daily_data.apply(np.real)

In [ ]:
wvlt_scaled_df = dwt_daily_data

In [ ]:
features = wvlt_scaled_df.iloc[:, :78]
targets = wvlt_scaled_df.iloc[:, 78:]

In [ ]:
targets

,HV,CCV,PV,GKV,RSV,YZV,GKYZV
Date,,,,,,,
2000-02-02,0.039337,0.039337,0.035334,0.034017,0.032740,0.282636,0.034017
2000-02-03,0.039588,0.039588,0.036011,0.034689,0.033467,0.285768,0.034689
2000-02-04,0.037904,0.037904,0.034207,0.032966,0.032057,0.279604,0.032966
2000-02-07,0.044257,0.044257,0.038502,0.036397,0.034853,0.291652,0.036397
2000-02-08,0.049006,0.049006,0.041434,0.037620,0.035254,0.293459,0.037620
...,...,...,...,...,...,...,...
2024-08-26,0.000123,0.000123,0.000011,0.000006,0.000006,0.000035,0.000065
2024-08-27,0.000656,0.000656,0.000029,-0.000005,-0.000017,-0.000063,0.000084
2024-08-28,0.000087,0.000087,0.000023,0.000028,0.000048,0.000235,0.000068


# Calculate the Correlation

In [ ]:
def clean_data(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    return df

## pearson Correlation

In [ ]:
feature_columns = features.columns
target_columns = targets.columns

In [ ]:
pearson_corr = wvlt_scaled_df.corr(method='pearson')
feature_target_pearson_corr = pearson_corr.loc[feature_columns, target_columns]

In [ ]:
feature_target_pearson_corr

,HV,CCV,PV,GKV,RSV,YZV,GKYZV
log_returns,0.014576,0.014576,0.012583,0.014309,0.014873,0.018058,0.015756
volume,0.815130,0.815130,0.820437,0.827481,0.828122,0.897392,0.837865
"('GARCH', 'normal', 0)",0.971580,0.971580,0.961330,0.956776,0.951597,0.905827,0.959819
"('GARCH', 'normal', 1)",0.946256,0.946256,0.937894,0.933207,0.928099,0.887146,0.935341
"('GARCH', 'gaussian', 0)",0.971580,0.971580,0.961330,0.956776,0.951597,0.905827,0.959819
...,...,...,...,...,...,...,...
usd_gbp,0.804834,0.804834,0.836056,0.839877,0.841002,0.951995,0.828814
usd_cny,0.819743,0.819743,0.849765,0.855390,0.857578,0.963821,0.849575
usd_cad,0.825832,0.825832,0.853040,0.859849,0.862584,0.964184,0.858328
usd_mxn,0.784538,0.784538,0.795678,0.806465,0.809725,0.919137,0.821855


In [ ]:
def get_high_correlation_features(correlation_df, target, threshold=0.5):
  high_corr_features = []

  target_corr = correlation_df[target]
  high_corr = target_corr[(target_corr > threshold) | (target_corr < -threshold)]

  for feature, value in high_corr.items():
    high_corr_features.append({
    'Target': target,
    'Feature': feature,
    'Correlation': value
    })

  high_corr_df = pd.DataFrame(high_corr_features)
  return high_corr_df

In [ ]:
highest_pearson_HV = get_high_correlation_features(feature_target_pearson_corr, 'HV')
highest_pearson_CCV = get_high_correlation_features(feature_target_pearson_corr, 'CCV')
highest_pearson_PV = get_high_correlation_features(feature_target_pearson_corr, 'PV')
highest_pearson_GKV = get_high_correlation_features(feature_target_pearson_corr, 'GKV')
highest_pearson_RSV = get_high_correlation_features(feature_target_pearson_corr, 'RSV')
highest_pearson_YZV = get_high_correlation_features(feature_target_pearson_corr, 'YZV')
highest_pearson_GKYZV = get_high_correlation_features(feature_target_pearson_corr, 'GKYZV')

In [ ]:
highest_pearson_HV

,Target,Feature,Correlation
0,HV,volume,0.815130
1,HV,"('GARCH', 'normal', 0)",0.971580
2,HV,"('GARCH', 'normal', 1)",0.946256
3,HV,"('GARCH', 'gaussian', 0)",0.971580
4,HV,"('GARCH', 'gaussian', 1)",0.946256
5,HV,"('GARCH', 'ged', 0)",0.969045
6,HV,"('GARCH', 'ged', 1)",0.941521
7,HV,"('FIGARCH', 'normal', 0)",0.971297
8,HV,"('FIGARCH', 'normal', 1)",0.971297
9,HV,"('FIGARCH', 'gaussian', 0)",0.971297


In [ ]:
highest_pearson_CCV

,Target,Feature,Correlation
0,CCV,volume,0.815130
1,CCV,"('GARCH', 'normal', 0)",0.971580
2,CCV,"('GARCH', 'normal', 1)",0.946256
3,CCV,"('GARCH', 'gaussian', 0)",0.971580
4,CCV,"('GARCH', 'gaussian', 1)",0.946256
5,CCV,"('GARCH', 'ged', 0)",0.969045
6,CCV,"('GARCH', 'ged', 1)",0.941521
7,CCV,"('FIGARCH', 'normal', 0)",0.971297
8,CCV,"('FIGARCH', 'normal', 1)",0.971297
9,CCV,"('FIGARCH', 'gaussian', 0)",0.971297


In [ ]:
highest_pearson_PV

,Target,Feature,Correlation
0,PV,volume,0.820437
1,PV,"('GARCH', 'normal', 0)",0.961330
2,PV,"('GARCH', 'normal', 1)",0.937894
3,PV,"('GARCH', 'gaussian', 0)",0.961330
4,PV,"('GARCH', 'gaussian', 1)",0.937894
5,PV,"('GARCH', 'ged', 0)",0.959766
6,PV,"('GARCH', 'ged', 1)",0.932264
7,PV,"('FIGARCH', 'normal', 0)",0.960565
8,PV,"('FIGARCH', 'normal', 1)",0.960565
9,PV,"('FIGARCH', 'gaussian', 0)",0.960565


In [ ]:
highest_pearson_GKV

,Target,Feature,Correlation
0,GKV,volume,0.827481
1,GKV,"('GARCH', 'normal', 0)",0.956776
2,GKV,"('GARCH', 'normal', 1)",0.933207
3,GKV,"('GARCH', 'gaussian', 0)",0.956776
4,GKV,"('GARCH', 'gaussian', 1)",0.933207
5,GKV,"('GARCH', 'ged', 0)",0.955208
6,GKV,"('GARCH', 'ged', 1)",0.927793
7,GKV,"('FIGARCH', 'normal', 0)",0.955900
8,GKV,"('FIGARCH', 'normal', 1)",0.955900
9,GKV,"('FIGARCH', 'gaussian', 0)",0.955900


In [ ]:
highest_pearson_RSV

,Target,Feature,Correlation
0,RSV,volume,0.828122
1,RSV,"('GARCH', 'normal', 0)",0.951597
2,RSV,"('GARCH', 'normal', 1)",0.928099
3,RSV,"('GARCH', 'gaussian', 0)",0.951597
4,RSV,"('GARCH', 'gaussian', 1)",0.928099
5,RSV,"('GARCH', 'ged', 0)",0.950121
6,RSV,"('GARCH', 'ged', 1)",0.922875
7,RSV,"('FIGARCH', 'normal', 0)",0.950499
8,RSV,"('FIGARCH', 'normal', 1)",0.950499
9,RSV,"('FIGARCH', 'gaussian', 0)",0.950499


In [ ]:
highest_pearson_YZV

,Target,Feature,Correlation
0,YZV,volume,0.897392
1,YZV,"('GARCH', 'normal', 0)",0.905827
2,YZV,"('GARCH', 'normal', 1)",0.887146
3,YZV,"('GARCH', 'gaussian', 0)",0.905827
4,YZV,"('GARCH', 'gaussian', 1)",0.887146
5,YZV,"('GARCH', 'ged', 0)",0.904220
6,YZV,"('GARCH', 'ged', 1)",0.880675
7,YZV,"('FIGARCH', 'normal', 0)",0.901510
8,YZV,"('FIGARCH', 'normal', 1)",0.901510
9,YZV,"('FIGARCH', 'gaussian', 0)",0.901510


In [ ]:
highest_pearson_GKYZV

,Target,Feature,Correlation
0,GKYZV,volume,0.837865
1,GKYZV,"('GARCH', 'normal', 0)",0.959819
2,GKYZV,"('GARCH', 'normal', 1)",0.935341
3,GKYZV,"('GARCH', 'gaussian', 0)",0.959819
4,GKYZV,"('GARCH', 'gaussian', 1)",0.935341
5,GKYZV,"('GARCH', 'ged', 0)",0.957475
6,GKYZV,"('GARCH', 'ged', 1)",0.930322
7,GKYZV,"('FIGARCH', 'normal', 0)",0.958923
8,GKYZV,"('FIGARCH', 'normal', 1)",0.958923
9,GKYZV,"('FIGARCH', 'gaussian', 0)",0.958923


## Spearman Correlation

In [ ]:
pearson_corr = wvlt_scaled_df.corr(method='spearman')
feature_target_spearman_corr = pearson_corr.loc[feature_columns, target_columns]

In [ ]:
feature_target_spearman_corr

,HV,CCV,PV,GKV,RSV,YZV,GKYZV
log_returns,0.080269,0.080269,0.099842,0.090541,0.079581,0.080378,0.095400
volume,0.380152,0.380152,0.415511,0.412933,0.400938,0.400891,0.415735
"('GARCH', 'normal', 0)",0.672547,0.672547,0.571680,0.472077,0.389458,0.393815,0.485259
"('GARCH', 'normal', 1)",0.596239,0.596239,0.499415,0.416517,0.351208,0.354600,0.425591
"('GARCH', 'gaussian', 0)",0.672547,0.672547,0.571680,0.472077,0.389458,0.393815,0.485259
...,...,...,...,...,...,...,...
usd_gbp,0.325712,0.325712,0.311676,0.311143,0.311130,0.311239,0.313354
usd_cny,0.325781,0.325781,0.346649,0.353991,0.360330,0.359923,0.353937
usd_cad,0.360567,0.360567,0.391437,0.404545,0.407600,0.406088,0.401464
usd_mxn,0.349239,0.349239,0.389595,0.406931,0.409656,0.409932,0.401482


In [ ]:
highest_spearman_HV = get_high_correlation_features(feature_target_spearman_corr, 'HV')
highest_spearman_CCV = get_high_correlation_features(feature_target_spearman_corr, 'CCV')
highest_spearman_PV = get_high_correlation_features(feature_target_spearman_corr, 'PV')
highest_spearman_GKV = get_high_correlation_features(feature_target_spearman_corr, 'GKV')
highest_spearman_RSV = get_high_correlation_features(feature_target_spearman_corr, 'RSV')
highest_spearman_YZV = get_high_correlation_features(feature_target_spearman_corr, 'YZV')
highest_spearman_GKYZV = get_high_correlation_features(feature_target_spearman_corr, 'GKYZV')

In [ ]:
highest_spearman_HV

,Target,Feature,Correlation
0,HV,"('GARCH', 'normal', 0)",0.672547
1,HV,"('GARCH', 'normal', 1)",0.596239
2,HV,"('GARCH', 'gaussian', 0)",0.672547
3,HV,"('GARCH', 'gaussian', 1)",0.596239
4,HV,"('GARCH', 'ged', 0)",0.630068
5,HV,"('GARCH', 'ged', 1)",0.564570
6,HV,"('FIGARCH', 'normal', 0)",0.725837
7,HV,"('FIGARCH', 'normal', 1)",0.725837
8,HV,"('FIGARCH', 'gaussian', 0)",0.725837
9,HV,"('FIGARCH', 'gaussian', 1)",0.725837


In [ ]:
highest_spearman_CCV

,Target,Feature,Correlation
0,CCV,"('GARCH', 'normal', 0)",0.672547
1,CCV,"('GARCH', 'normal', 1)",0.596239
2,CCV,"('GARCH', 'gaussian', 0)",0.672547
3,CCV,"('GARCH', 'gaussian', 1)",0.596239
4,CCV,"('GARCH', 'ged', 0)",0.630068
5,CCV,"('GARCH', 'ged', 1)",0.564570
6,CCV,"('FIGARCH', 'normal', 0)",0.725837
7,CCV,"('FIGARCH', 'normal', 1)",0.725837
8,CCV,"('FIGARCH', 'gaussian', 0)",0.725837
9,CCV,"('FIGARCH', 'gaussian', 1)",0.725837


In [ ]:
highest_spearman_PV

,Target,Feature,Correlation
0,PV,"('GARCH', 'normal', 0)",0.571680
1,PV,"('GARCH', 'gaussian', 0)",0.571680
2,PV,"('GARCH', 'ged', 0)",0.543192
3,PV,"('FIGARCH', 'normal', 0)",0.609557
4,PV,"('FIGARCH', 'normal', 1)",0.609557
5,PV,"('FIGARCH', 'gaussian', 0)",0.609557
6,PV,"('FIGARCH', 'gaussian', 1)",0.609557
7,PV,"('FIGARCH', 'ged', 0)",0.569641
8,PV,"('FIGARCH', 'ged', 1)",0.569641
9,PV,gdp_growth,0.525352


In [ ]:
highest_spearman_GKV

,Target,Feature,Correlation
0,GKV,gdp_growth,0.525103


In [ ]:
highest_spearman_RSV

,Target,Feature,Correlation
0,RSV,gdp_growth,0.524665


In [ ]:
highest_spearman_YZV

,Target,Feature,Correlation
0,YZV,gdp_growth,0.524535


In [ ]:
highest_spearman_GKYZV

,Target,Feature,Correlation
0,GKYZV,"('FIGARCH', 'normal', 0)",0.506668
1,GKYZV,"('FIGARCH', 'normal', 1)",0.506668
2,GKYZV,"('FIGARCH', 'gaussian', 0)",0.506668
3,GKYZV,"('FIGARCH', 'gaussian', 1)",0.506668
4,GKYZV,gdp_growth,0.526590


## Granger Causality

In [ ]:
def calculate_granger_causality(scaled_df, feature_columns, target, maxlag=5, significance_level=0.05):
    significant_features = []
    cleaned_df = clean_data(scaled_df)
    for feature in feature_columns:
        if feature not in cleaned_df.columns:
            continue
        try:
            test_result = grangercausalitytests(cleaned_df[[target, feature]], maxlag=maxlag, verbose=False)
            p_value = [test[0]['ssr_chi2test'][1] for test in test_result.values()]
            if max(p_value) < significance_level:
                significant_features.append({
                    'Target': target,
                    'Feature': feature,
                    'P-Value': p_value
                })
        except Exception as e:
            print(f"Error processing {feature} with target {target}: {e}")

    significant_features_df = pd.DataFrame(significant_features)
    return significant_features_df

In [ ]:
highest_granger_HV = calculate_granger_causality(wvlt_scaled_df, features.columns, 'HV')
highest_granger_CCV = calculate_granger_causality(wvlt_scaled_df, features.columns, 'CCV')
highest_granger_PV = calculate_granger_causality(wvlt_scaled_df, features.columns, 'PV')
highest_granger_GKV = calculate_granger_causality(wvlt_scaled_df, features.columns, 'GKV')
highest_granger_RSV = calculate_granger_causality(wvlt_scaled_df, features.columns, 'RSV')
highest_granger_YZV = calculate_granger_causality(wvlt_scaled_df, features.columns, 'YZV')
highest_granger_GKYZV = calculate_granger_causality(wvlt_scaled_df, features.columns, 'GKYZV')

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print resul

In [ ]:
highest_granger_HV

,Target,Feature,P-Value
0,HV,log_returns,"[4.392325623702399e-36, 5.972909280932005e-39,..."
1,HV,volume,"[1.4866777883618945e-76, 3.8465734023266584e-1..."
2,HV,"('GARCH', 'normal', 0)","[0.0, 7.791268338733226e-223, 7.54960609164447..."
3,HV,"('GARCH', 'normal', 1)","[0.0, 2.9588881494096404e-256, 1.0548391955940..."
4,HV,"('GARCH', 'gaussian', 0)","[0.0, 7.791268338733226e-223, 7.54960609164447..."
5,HV,"('GARCH', 'gaussian', 1)","[0.0, 2.9588881494096404e-256, 1.0548391955940..."
6,HV,"('GARCH', 'ged', 0)","[0.0, 3.18287661753625e-247, 1.9052693437923e-..."
7,HV,"('GARCH', 'ged', 1)","[0.0, 1.9715087310619533e-197, 5.2003688357265..."
8,HV,"('FIGARCH', 'normal', 0)","[0.0, 5.28546444496791e-197, 3.662902369156403..."
9,HV,"('FIGARCH', 'normal', 1)","[0.0, 5.28546444496791e-197, 3.662902369156403..."


In [ ]:
highest_granger_CCV

,Target,Feature,P-Value
0,CCV,log_returns,"[4.392325623705876e-36, 5.972909280914834e-39,..."
1,CCV,volume,"[1.4866777883606623e-76, 3.8464021216670556e-1..."
2,CCV,"('GARCH', 'normal', 0)","[0.0, 7.791268338857032e-223, 7.54960609170208..."
3,CCV,"('GARCH', 'normal', 1)","[0.0, 2.9588881494483907e-256, 1.0548391955996..."
4,CCV,"('GARCH', 'gaussian', 0)","[0.0, 7.791268338857032e-223, 7.54960609170208..."
5,CCV,"('GARCH', 'gaussian', 1)","[0.0, 2.9588881494483907e-256, 1.0548391955996..."
6,CCV,"('GARCH', 'ged', 0)","[0.0, 3.182876617577936e-247, 1.90526934380792..."
7,CCV,"('GARCH', 'ged', 1)","[0.0, 1.971508731083855e-197, 5.20036883574579..."
8,CCV,"('FIGARCH', 'normal', 0)","[0.0, 5.2854644450585456e-197, 3.6629023691851..."
9,CCV,"('FIGARCH', 'normal', 1)","[0.0, 5.2854644450585456e-197, 3.6629023691851..."


In [ ]:
highest_granger_PV

,Target,Feature,P-Value
0,PV,log_returns,"[2.2282031588411853e-42, 3.211081763679349e-45..."
1,PV,volume,"[1.8384272862615134e-60, 3.870241161120042e-93..."
2,PV,"('GARCH', 'normal', 0)","[2.262076086648618e-208, 6.1644795166721986e-1..."
3,PV,"('GARCH', 'normal', 1)","[8.562034559954598e-255, 3.525412602593142e-13..."
4,PV,"('GARCH', 'gaussian', 0)","[2.262076086648618e-208, 6.1644795166721986e-1..."
5,PV,"('GARCH', 'gaussian', 1)","[8.562034559954598e-255, 3.525412602593142e-13..."
6,PV,"('GARCH', 'ged', 0)","[2.838751406283721e-235, 1.6527362522535344e-1..."
7,PV,"('GARCH', 'ged', 1)","[3.374066184966278e-215, 1.677014136182402e-11..."
8,PV,"('FIGARCH', 'normal', 0)","[2.094514902191612e-161, 2.7230222089320525e-7..."
9,PV,"('FIGARCH', 'normal', 1)","[2.094514902191612e-161, 2.7230222089320525e-7..."


In [ ]:
highest_granger_GKV

,Target,Feature,P-Value
0,GKV,log_returns,"[7.588391270068177e-43, 1.4494879992577137e-44..."
1,GKV,volume,"[6.572813210023112e-65, 4.789022250269458e-97,..."
2,GKV,"('GARCH', 'normal', 0)","[9.016083034819973e-247, 7.467728277484494e-15..."
3,GKV,"('GARCH', 'normal', 1)","[1.3533211912114984e-285, 4.230223510988563e-1..."
4,GKV,"('GARCH', 'gaussian', 0)","[9.016083034819973e-247, 7.467728277484494e-15..."
5,GKV,"('GARCH', 'gaussian', 1)","[1.3533211912114984e-285, 4.230223510988563e-1..."
6,GKV,"('GARCH', 'ged', 0)","[2.1030953800542735e-269, 4.103710831181301e-1..."
7,GKV,"('GARCH', 'ged', 1)","[2.5295687552062943e-248, 9.431575468270737e-1..."
8,GKV,"('FIGARCH', 'normal', 0)","[6.4800230989506e-194, 3.014149253007082e-124,..."
9,GKV,"('FIGARCH', 'normal', 1)","[6.4800230989506e-194, 3.014149253007082e-124,..."


In [ ]:
highest_granger_RSV

,Target,Feature,P-Value
0,RSV,log_returns,"[2.753591052003781e-40, 2.9680156511285668e-40..."
1,RSV,volume,"[1.9146600478089303e-67, 1.0962063037507887e-9..."
2,RSV,"('GARCH', 'normal', 0)","[5.2856339763417114e-248, 9.403991479183931e-1..."
3,RSV,"('GARCH', 'normal', 1)","[1.005072009204067e-282, 3.725094416680323e-22..."
4,RSV,"('GARCH', 'gaussian', 0)","[5.2856339763417114e-248, 9.403991479183931e-1..."
5,RSV,"('GARCH', 'gaussian', 1)","[1.005072009204067e-282, 3.725094416680323e-22..."
6,RSV,"('GARCH', 'ged', 0)","[1.4852575097991097e-264, 2.3199726524086173e-..."
7,RSV,"('GARCH', 'ged', 1)","[5.328560487997962e-249, 1.0383349100512791e-1..."
8,RSV,"('FIGARCH', 'normal', 0)","[1.3076022284946062e-200, 1.9107576859395936e-..."
9,RSV,"('FIGARCH', 'normal', 1)","[1.3076022284946062e-200, 1.9107576859395936e-..."


In [ ]:
highest_granger_YZV

,Target,Feature,P-Value
0,YZV,log_returns,"[3.1749684631068527e-34, 6.943404762269532e-33..."
1,YZV,volume,"[4.209703313255557e-43, 2.3189800025167765e-47..."
2,YZV,"('GARCH', 'normal', 0)","[1.1927454196995166e-26, 6.840612899248756e-10..."
3,YZV,"('GARCH', 'normal', 1)","[4.330684549805143e-54, 2.5844087553846266e-79..."
4,YZV,"('GARCH', 'gaussian', 0)","[1.1927454196995166e-26, 6.840612899248756e-10..."
5,YZV,"('GARCH', 'gaussian', 1)","[4.330684549805143e-54, 2.5844087553846266e-79..."
6,YZV,"('GARCH', 'ged', 0)","[1.166903560604906e-31, 1.1369006737335946e-10..."
7,YZV,"('GARCH', 'ged', 1)","[9.24768325088933e-51, 3.2753806084235743e-80,..."
8,YZV,"('FIGARCH', 'normal', 0)","[2.4205203853044183e-23, 1.5936641512160578e-9..."
9,YZV,"('FIGARCH', 'normal', 1)","[2.4205203853044183e-23, 1.5936641512160578e-9..."


In [ ]:
highest_granger_GKYZV

,Target,Feature,P-Value
0,GKYZV,log_returns,"[5.531597774555617e-41, 1.1969269677040826e-41..."
1,GKYZV,volume,"[9.857879680177978e-74, 1.506639731147175e-109..."
2,GKYZV,"('GARCH', 'normal', 0)","[0.0, 2.9612425282515688e-176, 2.0111467251038..."
3,GKYZV,"('GARCH', 'normal', 1)","[0.0, 7.87254101554825e-215, 1.770611840635215..."
4,GKYZV,"('GARCH', 'gaussian', 0)","[0.0, 2.9612425282515688e-176, 2.0111467251038..."
5,GKYZV,"('GARCH', 'gaussian', 1)","[0.0, 7.87254101554825e-215, 1.770611840635215..."
6,GKYZV,"('GARCH', 'ged', 0)","[0.0, 1.4047225376558219e-188, 3.7723397598158..."
7,GKYZV,"('GARCH', 'ged', 1)","[7.036718016773611e-302, 8.493008060828085e-19..."
8,GKYZV,"('FIGARCH', 'normal', 0)","[1.5619348781768049e-260, 2.0310714284138767e-..."
9,GKYZV,"('FIGARCH', 'normal', 1)","[1.5619348781768049e-260, 2.0310714284138767e-..."


## Cross Correlation

In [ ]:
def calculate_cross_correlation(scaled_df, feature_columns, target, maxlag=30, threshold=0.5):
  cross_corr_results = []
  for feature in feature_columns:
    cross_corr = ccf(scaled_df[feature], scaled_df[target])[:maxlag]
    max_corr_value = max(abs(cross_corr))
    if (max_corr_value > threshold):
      cross_corr_results.append({
          'Target': target,
          'Feature': feature,
          'Cross-Correlation': max_corr_value
      })

  cross_corr_results_df = pd.DataFrame(cross_corr_results)
  return cross_corr_results_df

In [ ]:
highest_ccf_HV = calculate_cross_correlation(wvlt_scaled_df, features.columns, 'HV')
highest_ccf_CCV = calculate_cross_correlation(wvlt_scaled_df, features.columns, 'CCV')
highest_ccf_PV = calculate_cross_correlation(wvlt_scaled_df, features.columns, 'PV')
highest_ccf_GKV = calculate_cross_correlation(wvlt_scaled_df, features.columns, 'GKV')
highest_ccf_RSV = calculate_cross_correlation(wvlt_scaled_df, features.columns, 'RSV')
highest_ccf_YZV = calculate_cross_correlation(wvlt_scaled_df, features.columns, 'YZV')
highest_ccf_GKYZV = calculate_cross_correlation(wvlt_scaled_df, features.columns, 'GKYZV')

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [ ]:
highest_ccf_HV

,Target,Feature,Cross-Correlation
0,HV,volume,0.815130
1,HV,"('GARCH', 'normal', 0)",0.971580
2,HV,"('GARCH', 'normal', 1)",0.946256
3,HV,"('GARCH', 'gaussian', 0)",0.971580
4,HV,"('GARCH', 'gaussian', 1)",0.946256
5,HV,"('GARCH', 'ged', 0)",0.969045
6,HV,"('GARCH', 'ged', 1)",0.941521
7,HV,"('FIGARCH', 'normal', 0)",0.971297
8,HV,"('FIGARCH', 'normal', 1)",0.971297
9,HV,"('FIGARCH', 'gaussian', 0)",0.971297


In [ ]:
highest_ccf_CCV

,Target,Feature,Cross-Correlation
0,CCV,volume,0.815130
1,CCV,"('GARCH', 'normal', 0)",0.971580
2,CCV,"('GARCH', 'normal', 1)",0.946256
3,CCV,"('GARCH', 'gaussian', 0)",0.971580
4,CCV,"('GARCH', 'gaussian', 1)",0.946256
5,CCV,"('GARCH', 'ged', 0)",0.969045
6,CCV,"('GARCH', 'ged', 1)",0.941521
7,CCV,"('FIGARCH', 'normal', 0)",0.971297
8,CCV,"('FIGARCH', 'normal', 1)",0.971297
9,CCV,"('FIGARCH', 'gaussian', 0)",0.971297


In [ ]:
highest_ccf_PV

,Target,Feature,Cross-Correlation
0,PV,volume,0.820437
1,PV,"('GARCH', 'normal', 0)",0.961330
2,PV,"('GARCH', 'normal', 1)",0.937894
3,PV,"('GARCH', 'gaussian', 0)",0.961330
4,PV,"('GARCH', 'gaussian', 1)",0.937894
5,PV,"('GARCH', 'ged', 0)",0.959766
6,PV,"('GARCH', 'ged', 1)",0.932264
7,PV,"('FIGARCH', 'normal', 0)",0.960565
8,PV,"('FIGARCH', 'normal', 1)",0.960565
9,PV,"('FIGARCH', 'gaussian', 0)",0.960565


In [ ]:
highest_ccf_GKV

,Target,Feature,Cross-Correlation
0,GKV,volume,0.827481
1,GKV,"('GARCH', 'normal', 0)",0.956776
2,GKV,"('GARCH', 'normal', 1)",0.933207
3,GKV,"('GARCH', 'gaussian', 0)",0.956776
4,GKV,"('GARCH', 'gaussian', 1)",0.933207
5,GKV,"('GARCH', 'ged', 0)",0.955208
6,GKV,"('GARCH', 'ged', 1)",0.927793
7,GKV,"('FIGARCH', 'normal', 0)",0.955900
8,GKV,"('FIGARCH', 'normal', 1)",0.955900
9,GKV,"('FIGARCH', 'gaussian', 0)",0.955900


In [ ]:
highest_ccf_RSV

,Target,Feature,Cross-Correlation
0,RSV,volume,0.828122
1,RSV,"('GARCH', 'normal', 0)",0.951597
2,RSV,"('GARCH', 'normal', 1)",0.928099
3,RSV,"('GARCH', 'gaussian', 0)",0.951597
4,RSV,"('GARCH', 'gaussian', 1)",0.928099
5,RSV,"('GARCH', 'ged', 0)",0.950121
6,RSV,"('GARCH', 'ged', 1)",0.922875
7,RSV,"('FIGARCH', 'normal', 0)",0.950499
8,RSV,"('FIGARCH', 'normal', 1)",0.950499
9,RSV,"('FIGARCH', 'gaussian', 0)",0.950499


In [ ]:
highest_ccf_YZV

,Target,Feature,Cross-Correlation
0,YZV,volume,0.897392
1,YZV,"('GARCH', 'normal', 0)",0.905827
2,YZV,"('GARCH', 'normal', 1)",0.887146
3,YZV,"('GARCH', 'gaussian', 0)",0.905827
4,YZV,"('GARCH', 'gaussian', 1)",0.887146
5,YZV,"('GARCH', 'ged', 0)",0.904220
6,YZV,"('GARCH', 'ged', 1)",0.880675
7,YZV,"('FIGARCH', 'normal', 0)",0.901510
8,YZV,"('FIGARCH', 'normal', 1)",0.901510
9,YZV,"('FIGARCH', 'gaussian', 0)",0.901510


In [ ]:
highest_ccf_GKYZV

,Target,Feature,Cross-Correlation
0,GKYZV,volume,0.837865
1,GKYZV,"('GARCH', 'normal', 0)",0.959819
2,GKYZV,"('GARCH', 'normal', 1)",0.935341
3,GKYZV,"('GARCH', 'gaussian', 0)",0.959819
4,GKYZV,"('GARCH', 'gaussian', 1)",0.935341
5,GKYZV,"('GARCH', 'ged', 0)",0.957475
6,GKYZV,"('GARCH', 'ged', 1)",0.930322
7,GKYZV,"('FIGARCH', 'normal', 0)",0.958923
8,GKYZV,"('FIGARCH', 'normal', 1)",0.958923
9,GKYZV,"('FIGARCH', 'gaussian', 0)",0.958923


## Results

### HV

In [ ]:
highest_pearson_HV

,Target,Feature,Correlation
0,HV,volume,0.815130
1,HV,"('GARCH', 'normal', 0)",0.971580
2,HV,"('GARCH', 'normal', 1)",0.946256
3,HV,"('GARCH', 'gaussian', 0)",0.971580
4,HV,"('GARCH', 'gaussian', 1)",0.946256
5,HV,"('GARCH', 'ged', 0)",0.969045
6,HV,"('GARCH', 'ged', 1)",0.941521
7,HV,"('FIGARCH', 'normal', 0)",0.971297
8,HV,"('FIGARCH', 'normal', 1)",0.971297
9,HV,"('FIGARCH', 'gaussian', 0)",0.971297


In [ ]:
highest_spearman_HV

,Target,Feature,Correlation
0,HV,"('GARCH', 'normal', 0)",0.672547
1,HV,"('GARCH', 'normal', 1)",0.596239
2,HV,"('GARCH', 'gaussian', 0)",0.672547
3,HV,"('GARCH', 'gaussian', 1)",0.596239
4,HV,"('GARCH', 'ged', 0)",0.630068
5,HV,"('GARCH', 'ged', 1)",0.564570
6,HV,"('FIGARCH', 'normal', 0)",0.725837
7,HV,"('FIGARCH', 'normal', 1)",0.725837
8,HV,"('FIGARCH', 'gaussian', 0)",0.725837
9,HV,"('FIGARCH', 'gaussian', 1)",0.725837


In [ ]:
highest_granger_HV

,Target,Feature,P-Value
0,HV,log_returns,"[4.392325623702399e-36, 5.972909280932005e-39,..."
1,HV,volume,"[1.4866777883618945e-76, 3.8465734023266584e-1..."
2,HV,"('GARCH', 'normal', 0)","[0.0, 7.791268338733226e-223, 7.54960609164447..."
3,HV,"('GARCH', 'normal', 1)","[0.0, 2.9588881494096404e-256, 1.0548391955940..."
4,HV,"('GARCH', 'gaussian', 0)","[0.0, 7.791268338733226e-223, 7.54960609164447..."
5,HV,"('GARCH', 'gaussian', 1)","[0.0, 2.9588881494096404e-256, 1.0548391955940..."
6,HV,"('GARCH', 'ged', 0)","[0.0, 3.18287661753625e-247, 1.9052693437923e-..."
7,HV,"('GARCH', 'ged', 1)","[0.0, 1.9715087310619533e-197, 5.2003688357265..."
8,HV,"('FIGARCH', 'normal', 0)","[0.0, 5.28546444496791e-197, 3.662902369156403..."
9,HV,"('FIGARCH', 'normal', 1)","[0.0, 5.28546444496791e-197, 3.662902369156403..."


In [ ]:
highest_ccf_HV

,Target,Feature,Cross-Correlation
0,HV,volume,0.815130
1,HV,"('GARCH', 'normal', 0)",0.971580
2,HV,"('GARCH', 'normal', 1)",0.946256
3,HV,"('GARCH', 'gaussian', 0)",0.971580
4,HV,"('GARCH', 'gaussian', 1)",0.946256
5,HV,"('GARCH', 'ged', 0)",0.969045
6,HV,"('GARCH', 'ged', 1)",0.941521
7,HV,"('FIGARCH', 'normal', 0)",0.971297
8,HV,"('FIGARCH', 'normal', 1)",0.971297
9,HV,"('FIGARCH', 'gaussian', 0)",0.971297


### CCV

In [ ]:
highest_pearson_CCV

,Target,Feature,Correlation
0,CCV,volume,0.815130
1,CCV,"('GARCH', 'normal', 0)",0.971580
2,CCV,"('GARCH', 'normal', 1)",0.946256
3,CCV,"('GARCH', 'gaussian', 0)",0.971580
4,CCV,"('GARCH', 'gaussian', 1)",0.946256
5,CCV,"('GARCH', 'ged', 0)",0.969045
6,CCV,"('GARCH', 'ged', 1)",0.941521
7,CCV,"('FIGARCH', 'normal', 0)",0.971297
8,CCV,"('FIGARCH', 'normal', 1)",0.971297
9,CCV,"('FIGARCH', 'gaussian', 0)",0.971297


In [ ]:
highest_spearman_CCV

,Target,Feature,Correlation
0,CCV,"('GARCH', 'normal', 0)",0.672547
1,CCV,"('GARCH', 'normal', 1)",0.596239
2,CCV,"('GARCH', 'gaussian', 0)",0.672547
3,CCV,"('GARCH', 'gaussian', 1)",0.596239
4,CCV,"('GARCH', 'ged', 0)",0.630068
5,CCV,"('GARCH', 'ged', 1)",0.564570
6,CCV,"('FIGARCH', 'normal', 0)",0.725837
7,CCV,"('FIGARCH', 'normal', 1)",0.725837
8,CCV,"('FIGARCH', 'gaussian', 0)",0.725837
9,CCV,"('FIGARCH', 'gaussian', 1)",0.725837


In [ ]:
highest_granger_CCV

,Target,Feature,P-Value
0,CCV,log_returns,"[4.392325623705876e-36, 5.972909280914834e-39,..."
1,CCV,volume,"[1.4866777883606623e-76, 3.8464021216670556e-1..."
2,CCV,"('GARCH', 'normal', 0)","[0.0, 7.791268338857032e-223, 7.54960609170208..."
3,CCV,"('GARCH', 'normal', 1)","[0.0, 2.9588881494483907e-256, 1.0548391955996..."
4,CCV,"('GARCH', 'gaussian', 0)","[0.0, 7.791268338857032e-223, 7.54960609170208..."
5,CCV,"('GARCH', 'gaussian', 1)","[0.0, 2.9588881494483907e-256, 1.0548391955996..."
6,CCV,"('GARCH', 'ged', 0)","[0.0, 3.182876617577936e-247, 1.90526934380792..."
7,CCV,"('GARCH', 'ged', 1)","[0.0, 1.971508731083855e-197, 5.20036883574579..."
8,CCV,"('FIGARCH', 'normal', 0)","[0.0, 5.2854644450585456e-197, 3.6629023691851..."
9,CCV,"('FIGARCH', 'normal', 1)","[0.0, 5.2854644450585456e-197, 3.6629023691851..."


In [ ]:
highest_ccf_CCV

,Target,Feature,Cross-Correlation
0,CCV,volume,0.815130
1,CCV,"('GARCH', 'normal', 0)",0.971580
2,CCV,"('GARCH', 'normal', 1)",0.946256
3,CCV,"('GARCH', 'gaussian', 0)",0.971580
4,CCV,"('GARCH', 'gaussian', 1)",0.946256
5,CCV,"('GARCH', 'ged', 0)",0.969045
6,CCV,"('GARCH', 'ged', 1)",0.941521
7,CCV,"('FIGARCH', 'normal', 0)",0.971297
8,CCV,"('FIGARCH', 'normal', 1)",0.971297
9,CCV,"('FIGARCH', 'gaussian', 0)",0.971297


### PV

In [ ]:
highest_pearson_PV

,Target,Feature,Correlation
0,PV,volume,0.820437
1,PV,"('GARCH', 'normal', 0)",0.961330
2,PV,"('GARCH', 'normal', 1)",0.937894
3,PV,"('GARCH', 'gaussian', 0)",0.961330
4,PV,"('GARCH', 'gaussian', 1)",0.937894
5,PV,"('GARCH', 'ged', 0)",0.959766
6,PV,"('GARCH', 'ged', 1)",0.932264
7,PV,"('FIGARCH', 'normal', 0)",0.960565
8,PV,"('FIGARCH', 'normal', 1)",0.960565
9,PV,"('FIGARCH', 'gaussian', 0)",0.960565


In [ ]:
highest_spearman_PV

,Target,Feature,Correlation
0,PV,"('GARCH', 'normal', 0)",0.571680
1,PV,"('GARCH', 'gaussian', 0)",0.571680
2,PV,"('GARCH', 'ged', 0)",0.543192
3,PV,"('FIGARCH', 'normal', 0)",0.609557
4,PV,"('FIGARCH', 'normal', 1)",0.609557
5,PV,"('FIGARCH', 'gaussian', 0)",0.609557
6,PV,"('FIGARCH', 'gaussian', 1)",0.609557
7,PV,"('FIGARCH', 'ged', 0)",0.569641
8,PV,"('FIGARCH', 'ged', 1)",0.569641
9,PV,gdp_growth,0.525352


In [ ]:
highest_granger_PV

,Target,Feature,P-Value
0,PV,log_returns,"[2.2282031588411853e-42, 3.211081763679349e-45..."
1,PV,volume,"[1.8384272862615134e-60, 3.870241161120042e-93..."
2,PV,"('GARCH', 'normal', 0)","[2.262076086648618e-208, 6.1644795166721986e-1..."
3,PV,"('GARCH', 'normal', 1)","[8.562034559954598e-255, 3.525412602593142e-13..."
4,PV,"('GARCH', 'gaussian', 0)","[2.262076086648618e-208, 6.1644795166721986e-1..."
5,PV,"('GARCH', 'gaussian', 1)","[8.562034559954598e-255, 3.525412602593142e-13..."
6,PV,"('GARCH', 'ged', 0)","[2.838751406283721e-235, 1.6527362522535344e-1..."
7,PV,"('GARCH', 'ged', 1)","[3.374066184966278e-215, 1.677014136182402e-11..."
8,PV,"('FIGARCH', 'normal', 0)","[2.094514902191612e-161, 2.7230222089320525e-7..."
9,PV,"('FIGARCH', 'normal', 1)","[2.094514902191612e-161, 2.7230222089320525e-7..."


In [ ]:
highest_ccf_PV

,Target,Feature,Cross-Correlation
0,PV,volume,0.820437
1,PV,"('GARCH', 'normal', 0)",0.961330
2,PV,"('GARCH', 'normal', 1)",0.937894
3,PV,"('GARCH', 'gaussian', 0)",0.961330
4,PV,"('GARCH', 'gaussian', 1)",0.937894
5,PV,"('GARCH', 'ged', 0)",0.959766
6,PV,"('GARCH', 'ged', 1)",0.932264
7,PV,"('FIGARCH', 'normal', 0)",0.960565
8,PV,"('FIGARCH', 'normal', 1)",0.960565
9,PV,"('FIGARCH', 'gaussian', 0)",0.960565


### GKV

In [ ]:
highest_pearson_GKV

,Target,Feature,Correlation
0,GKV,volume,0.827481
1,GKV,"('GARCH', 'normal', 0)",0.956776
2,GKV,"('GARCH', 'normal', 1)",0.933207
3,GKV,"('GARCH', 'gaussian', 0)",0.956776
4,GKV,"('GARCH', 'gaussian', 1)",0.933207
5,GKV,"('GARCH', 'ged', 0)",0.955208
6,GKV,"('GARCH', 'ged', 1)",0.927793
7,GKV,"('FIGARCH', 'normal', 0)",0.955900
8,GKV,"('FIGARCH', 'normal', 1)",0.955900
9,GKV,"('FIGARCH', 'gaussian', 0)",0.955900


In [ ]:
highest_spearman_GKV

,Target,Feature,Correlation
0,GKV,gdp_growth,0.525103


In [ ]:
highest_granger_GKV

,Target,Feature,P-Value
0,GKV,log_returns,"[7.588391270068177e-43, 1.4494879992577137e-44..."
1,GKV,volume,"[6.572813210023112e-65, 4.789022250269458e-97,..."
2,GKV,"('GARCH', 'normal', 0)","[9.016083034819973e-247, 7.467728277484494e-15..."
3,GKV,"('GARCH', 'normal', 1)","[1.3533211912114984e-285, 4.230223510988563e-1..."
4,GKV,"('GARCH', 'gaussian', 0)","[9.016083034819973e-247, 7.467728277484494e-15..."
5,GKV,"('GARCH', 'gaussian', 1)","[1.3533211912114984e-285, 4.230223510988563e-1..."
6,GKV,"('GARCH', 'ged', 0)","[2.1030953800542735e-269, 4.103710831181301e-1..."
7,GKV,"('GARCH', 'ged', 1)","[2.5295687552062943e-248, 9.431575468270737e-1..."
8,GKV,"('FIGARCH', 'normal', 0)","[6.4800230989506e-194, 3.014149253007082e-124,..."
9,GKV,"('FIGARCH', 'normal', 1)","[6.4800230989506e-194, 3.014149253007082e-124,..."


In [ ]:
highest_ccf_GKV

,Target,Feature,Cross-Correlation
0,GKV,volume,0.827481
1,GKV,"('GARCH', 'normal', 0)",0.956776
2,GKV,"('GARCH', 'normal', 1)",0.933207
3,GKV,"('GARCH', 'gaussian', 0)",0.956776
4,GKV,"('GARCH', 'gaussian', 1)",0.933207
5,GKV,"('GARCH', 'ged', 0)",0.955208
6,GKV,"('GARCH', 'ged', 1)",0.927793
7,GKV,"('FIGARCH', 'normal', 0)",0.955900
8,GKV,"('FIGARCH', 'normal', 1)",0.955900
9,GKV,"('FIGARCH', 'gaussian', 0)",0.955900


### RSV

In [ ]:
highest_pearson_RSV

,Target,Feature,Correlation
0,RSV,volume,0.828122
1,RSV,"('GARCH', 'normal', 0)",0.951597
2,RSV,"('GARCH', 'normal', 1)",0.928099
3,RSV,"('GARCH', 'gaussian', 0)",0.951597
4,RSV,"('GARCH', 'gaussian', 1)",0.928099
5,RSV,"('GARCH', 'ged', 0)",0.950121
6,RSV,"('GARCH', 'ged', 1)",0.922875
7,RSV,"('FIGARCH', 'normal', 0)",0.950499
8,RSV,"('FIGARCH', 'normal', 1)",0.950499
9,RSV,"('FIGARCH', 'gaussian', 0)",0.950499


In [ ]:
highest_spearman_RSV

,Target,Feature,Correlation
0,RSV,gdp_growth,0.524665


In [ ]:
highest_granger_RSV

,Target,Feature,P-Value
0,RSV,log_returns,"[2.753591052003781e-40, 2.9680156511285668e-40..."
1,RSV,volume,"[1.9146600478089303e-67, 1.0962063037507887e-9..."
2,RSV,"('GARCH', 'normal', 0)","[5.2856339763417114e-248, 9.403991479183931e-1..."
3,RSV,"('GARCH', 'normal', 1)","[1.005072009204067e-282, 3.725094416680323e-22..."
4,RSV,"('GARCH', 'gaussian', 0)","[5.2856339763417114e-248, 9.403991479183931e-1..."
5,RSV,"('GARCH', 'gaussian', 1)","[1.005072009204067e-282, 3.725094416680323e-22..."
6,RSV,"('GARCH', 'ged', 0)","[1.4852575097991097e-264, 2.3199726524086173e-..."
7,RSV,"('GARCH', 'ged', 1)","[5.328560487997962e-249, 1.0383349100512791e-1..."
8,RSV,"('FIGARCH', 'normal', 0)","[1.3076022284946062e-200, 1.9107576859395936e-..."
9,RSV,"('FIGARCH', 'normal', 1)","[1.3076022284946062e-200, 1.9107576859395936e-..."


In [ ]:
highest_ccf_RSV

,Target,Feature,Cross-Correlation
0,RSV,volume,0.828122
1,RSV,"('GARCH', 'normal', 0)",0.951597
2,RSV,"('GARCH', 'normal', 1)",0.928099
3,RSV,"('GARCH', 'gaussian', 0)",0.951597
4,RSV,"('GARCH', 'gaussian', 1)",0.928099
5,RSV,"('GARCH', 'ged', 0)",0.950121
6,RSV,"('GARCH', 'ged', 1)",0.922875
7,RSV,"('FIGARCH', 'normal', 0)",0.950499
8,RSV,"('FIGARCH', 'normal', 1)",0.950499
9,RSV,"('FIGARCH', 'gaussian', 0)",0.950499


### YZV

In [ ]:
highest_pearson_YZV

,Target,Feature,Correlation
0,YZV,volume,0.897392
1,YZV,"('GARCH', 'normal', 0)",0.905827
2,YZV,"('GARCH', 'normal', 1)",0.887146
3,YZV,"('GARCH', 'gaussian', 0)",0.905827
4,YZV,"('GARCH', 'gaussian', 1)",0.887146
5,YZV,"('GARCH', 'ged', 0)",0.904220
6,YZV,"('GARCH', 'ged', 1)",0.880675
7,YZV,"('FIGARCH', 'normal', 0)",0.901510
8,YZV,"('FIGARCH', 'normal', 1)",0.901510
9,YZV,"('FIGARCH', 'gaussian', 0)",0.901510


In [ ]:
highest_spearman_YZV

,Target,Feature,Correlation
0,YZV,gdp_growth,0.524535


In [ ]:
highest_granger_YZV

,Target,Feature,P-Value
0,YZV,log_returns,"[3.1749684631068527e-34, 6.943404762269532e-33..."
1,YZV,volume,"[4.209703313255557e-43, 2.3189800025167765e-47..."
2,YZV,"('GARCH', 'normal', 0)","[1.1927454196995166e-26, 6.840612899248756e-10..."
3,YZV,"('GARCH', 'normal', 1)","[4.330684549805143e-54, 2.5844087553846266e-79..."
4,YZV,"('GARCH', 'gaussian', 0)","[1.1927454196995166e-26, 6.840612899248756e-10..."
5,YZV,"('GARCH', 'gaussian', 1)","[4.330684549805143e-54, 2.5844087553846266e-79..."
6,YZV,"('GARCH', 'ged', 0)","[1.166903560604906e-31, 1.1369006737335946e-10..."
7,YZV,"('GARCH', 'ged', 1)","[9.24768325088933e-51, 3.2753806084235743e-80,..."
8,YZV,"('FIGARCH', 'normal', 0)","[2.4205203853044183e-23, 1.5936641512160578e-9..."
9,YZV,"('FIGARCH', 'normal', 1)","[2.4205203853044183e-23, 1.5936641512160578e-9..."


In [ ]:
highest_ccf_YZV

,Target,Feature,Cross-Correlation
0,YZV,volume,0.897392
1,YZV,"('GARCH', 'normal', 0)",0.905827
2,YZV,"('GARCH', 'normal', 1)",0.887146
3,YZV,"('GARCH', 'gaussian', 0)",0.905827
4,YZV,"('GARCH', 'gaussian', 1)",0.887146
5,YZV,"('GARCH', 'ged', 0)",0.904220
6,YZV,"('GARCH', 'ged', 1)",0.880675
7,YZV,"('FIGARCH', 'normal', 0)",0.901510
8,YZV,"('FIGARCH', 'normal', 1)",0.901510
9,YZV,"('FIGARCH', 'gaussian', 0)",0.901510


### GKYZV

In [ ]:
highest_pearson_GKYZV

,Target,Feature,Correlation
0,GKYZV,volume,0.837865
1,GKYZV,"('GARCH', 'normal', 0)",0.959819
2,GKYZV,"('GARCH', 'normal', 1)",0.935341
3,GKYZV,"('GARCH', 'gaussian', 0)",0.959819
4,GKYZV,"('GARCH', 'gaussian', 1)",0.935341
5,GKYZV,"('GARCH', 'ged', 0)",0.957475
6,GKYZV,"('GARCH', 'ged', 1)",0.930322
7,GKYZV,"('FIGARCH', 'normal', 0)",0.958923
8,GKYZV,"('FIGARCH', 'normal', 1)",0.958923
9,GKYZV,"('FIGARCH', 'gaussian', 0)",0.958923


In [ ]:
highest_spearman_GKYZV

,Target,Feature,Correlation
0,GKYZV,"('FIGARCH', 'normal', 0)",0.506668
1,GKYZV,"('FIGARCH', 'normal', 1)",0.506668
2,GKYZV,"('FIGARCH', 'gaussian', 0)",0.506668
3,GKYZV,"('FIGARCH', 'gaussian', 1)",0.506668
4,GKYZV,gdp_growth,0.526590


In [ ]:
highest_granger_GKYZV

,Target,Feature,P-Value
0,GKYZV,log_returns,"[5.531597774555617e-41, 1.1969269677040826e-41..."
1,GKYZV,volume,"[9.857879680177978e-74, 1.506639731147175e-109..."
2,GKYZV,"('GARCH', 'normal', 0)","[0.0, 2.9612425282515688e-176, 2.0111467251038..."
3,GKYZV,"('GARCH', 'normal', 1)","[0.0, 7.87254101554825e-215, 1.770611840635215..."
4,GKYZV,"('GARCH', 'gaussian', 0)","[0.0, 2.9612425282515688e-176, 2.0111467251038..."
5,GKYZV,"('GARCH', 'gaussian', 1)","[0.0, 7.87254101554825e-215, 1.770611840635215..."
6,GKYZV,"('GARCH', 'ged', 0)","[0.0, 1.4047225376558219e-188, 3.7723397598158..."
7,GKYZV,"('GARCH', 'ged', 1)","[7.036718016773611e-302, 8.493008060828085e-19..."
8,GKYZV,"('FIGARCH', 'normal', 0)","[1.5619348781768049e-260, 2.0310714284138767e-..."
9,GKYZV,"('FIGARCH', 'normal', 1)","[1.5619348781768049e-260, 2.0310714284138767e-..."


In [ ]:
highest_ccf_GKYZV

,Target,Feature,Cross-Correlation
0,GKYZV,volume,0.837865
1,GKYZV,"('GARCH', 'normal', 0)",0.959819
2,GKYZV,"('GARCH', 'normal', 1)",0.935341
3,GKYZV,"('GARCH', 'gaussian', 0)",0.959819
4,GKYZV,"('GARCH', 'gaussian', 1)",0.935341
5,GKYZV,"('GARCH', 'ged', 0)",0.957475
6,GKYZV,"('GARCH', 'ged', 1)",0.930322
7,GKYZV,"('FIGARCH', 'normal', 0)",0.958923
8,GKYZV,"('FIGARCH', 'normal', 1)",0.958923
9,GKYZV,"('FIGARCH', 'gaussian', 0)",0.958923


# Combine criteria


In [ ]:
def concat_non_empty_dfs(*dfs):
    non_empty_dfs = [df[['Feature']] for df in dfs if 'Feature' in df.columns and not df.empty]
    if non_empty_dfs:
        return pd.concat(non_empty_dfs, axis=0, ignore_index=True).drop_duplicates()
    else:
        return pd.DataFrame()  # Return an empty DataFrame if all are empty

In [ ]:
HV_features = concat_non_empty_dfs(highest_spearman_HV, highest_pearson_HV, highest_granger_HV, highest_ccf_HV)
CCV_features = concat_non_empty_dfs(highest_spearman_CCV, highest_pearson_CCV, highest_granger_CCV, highest_ccf_CCV)
PV_features = concat_non_empty_dfs(highest_spearman_PV, highest_pearson_PV, highest_granger_PV, highest_ccf_PV)
GKV_features = concat_non_empty_dfs(highest_spearman_GKV, highest_pearson_GKV, highest_granger_GKV, highest_ccf_GKV)
RSV_features = concat_non_empty_dfs(highest_spearman_RSV, highest_pearson_RSV, highest_granger_RSV, highest_ccf_RSV)
YZV_features = concat_non_empty_dfs(highest_spearman_YZV, highest_pearson_YZV, highest_granger_YZV, highest_ccf_YZV)
GKYZV_features = concat_non_empty_dfs(highest_spearman_GKYZV, highest_pearson_GKYZV, highest_granger_GKYZV, highest_ccf_GKYZV)

In [ ]:
def load_data(target, features_list, features_df, targets):
  data = pd.DataFrame(index=targets.index)
  data[target] = targets[target].values

  for feature in features_list.values:
    data[feature] = features_df[feature].values

  return data

In [ ]:
HV_data = load_data('HV', HV_features, features, targets)
CCV_data = load_data('CCV', CCV_features, features, targets)
PV_data = load_data('PV', PV_features, features, targets)
GKV_data = load_data('GKV', GKV_features, features, targets)
RSV_data = load_data('RSV', RSV_features, features, targets)
YZV_data = load_data('YZV', YZV_features, features, targets)
GKYZV_data = load_data('GKYZV', GKYZV_features, features, targets)

In [ ]:
HV_data.index = pd.to_datetime(HV_data.index)
CCV_data.index = pd.to_datetime(CCV_data.index)
PV_data.index = pd.to_datetime(PV_data.index)
GKV_data.index = pd.to_datetime(GKV_data.index)
RSV_data.index = pd.to_datetime(RSV_data.index)
YZV_data.index = pd.to_datetime(YZV_data.index)
GKYZV_data.index = pd.to_datetime(GKYZV_data.index)

In [ ]:
path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

In [ ]:
HV_data.columns

Index(['HV', '('GARCH', 'normal', 0)', '('GARCH', 'normal', 1)',
       '('GARCH', 'gaussian', 0)', '('GARCH', 'gaussian', 1)',
       '('GARCH', 'ged', 0)', '('GARCH', 'ged', 1)',
       '('FIGARCH', 'normal', 0)', '('FIGARCH', 'normal', 1)',
       '('FIGARCH', 'gaussian', 0)', '('FIGARCH', 'gaussian', 1)',
       '('FIGARCH', 'ged', 0)', '('FIGARCH', 'ged', 1)',
       '('APARCH', 'normal', 0)', '('APARCH', 'normal', 1)',
       '('APARCH', 'gaussian', 0)', '('APARCH', 'gaussian', 1)',
       '('APARCH', 'ged', 0)', 'gdp_growth', 'volume', 'oil', 'gold', 'nasdaq',
       'djia', 'vix', 'interest_rate', 'inflation_rate', 'unemployment_rate',
       'usd_eur', 'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn',
       'gt_data', 'log_returns'],
      dtype='object')

In [ ]:
CCV_data.columns

Index(['CCV', '('GARCH', 'normal', 0)', '('GARCH', 'normal', 1)',
       '('GARCH', 'gaussian', 0)', '('GARCH', 'gaussian', 1)',
       '('GARCH', 'ged', 0)', '('GARCH', 'ged', 1)',
       '('FIGARCH', 'normal', 0)', '('FIGARCH', 'normal', 1)',
       '('FIGARCH', 'gaussian', 0)', '('FIGARCH', 'gaussian', 1)',
       '('FIGARCH', 'ged', 0)', '('FIGARCH', 'ged', 1)',
       '('APARCH', 'normal', 0)', '('APARCH', 'normal', 1)',
       '('APARCH', 'gaussian', 0)', '('APARCH', 'gaussian', 1)',
       '('APARCH', 'ged', 0)', 'gdp_growth', 'volume', 'oil', 'gold', 'nasdaq',
       'djia', 'vix', 'interest_rate', 'inflation_rate', 'unemployment_rate',
       'usd_eur', 'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn',
       'gt_data', 'log_returns'],
      dtype='object')

In [ ]:
PV_data.columns

Index(['PV', '('GARCH', 'normal', 0)', '('GARCH', 'gaussian', 0)',
       '('GARCH', 'ged', 0)', '('FIGARCH', 'normal', 0)',
       '('FIGARCH', 'normal', 1)', '('FIGARCH', 'gaussian', 0)',
       '('FIGARCH', 'gaussian', 1)', '('FIGARCH', 'ged', 0)',
       '('FIGARCH', 'ged', 1)', 'gdp_growth', 'volume',
       '('GARCH', 'normal', 1)', '('GARCH', 'gaussian', 1)',
       '('GARCH', 'ged', 1)', 'oil', 'gold', 'nasdaq', 'djia', 'vix',
       'interest_rate', 'inflation_rate', 'unemployment_rate', 'usd_eur',
       'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn', 'gt_data',
       'log_returns', '('APARCH', 'studentst', 1)'],
      dtype='object')

In [ ]:
GKV_data.columns

Index(['GKV', 'gdp_growth', 'volume', '('GARCH', 'normal', 0)',
       '('GARCH', 'normal', 1)', '('GARCH', 'gaussian', 0)',
       '('GARCH', 'gaussian', 1)', '('GARCH', 'ged', 0)',
       '('GARCH', 'ged', 1)', '('FIGARCH', 'normal', 0)',
       '('FIGARCH', 'normal', 1)', '('FIGARCH', 'gaussian', 0)',
       '('FIGARCH', 'gaussian', 1)', 'oil', 'gold', 'nasdaq', 'djia', 'vix',
       'interest_rate', 'inflation_rate', 'unemployment_rate', 'usd_eur',
       'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn', 'gt_data',
       'log_returns', '('APARCH', 'studentst', 1)'],
      dtype='object')

In [ ]:
RSV_data.columns

Index(['RSV', 'gdp_growth', 'volume', '('GARCH', 'normal', 0)',
       '('GARCH', 'normal', 1)', '('GARCH', 'gaussian', 0)',
       '('GARCH', 'gaussian', 1)', '('GARCH', 'ged', 0)',
       '('GARCH', 'ged', 1)', '('FIGARCH', 'normal', 0)',
       '('FIGARCH', 'normal', 1)', '('FIGARCH', 'gaussian', 0)',
       '('FIGARCH', 'gaussian', 1)', 'oil', 'gold', 'nasdaq', 'djia', 'vix',
       'interest_rate', 'inflation_rate', 'unemployment_rate', 'usd_eur',
       'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn', 'gt_data',
       'log_returns', '('APARCH', 'studentst', 1)'],
      dtype='object')

In [ ]:
YZV_data.columns

Index(['YZV', 'gdp_growth', 'volume', '('GARCH', 'normal', 0)',
       '('GARCH', 'normal', 1)', '('GARCH', 'gaussian', 0)',
       '('GARCH', 'gaussian', 1)', '('GARCH', 'ged', 0)',
       '('GARCH', 'ged', 1)', '('FIGARCH', 'normal', 0)',
       '('FIGARCH', 'normal', 1)', '('FIGARCH', 'gaussian', 0)',
       '('FIGARCH', 'gaussian', 1)', 'oil', 'gold', 'nasdaq', 'djia', 'vix',
       'interest_rate', 'inflation_rate', 'unemployment_rate', 'usd_eur',
       'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn', 'gt_data',
       'log_returns', '('APARCH', 'studentst', 1)'],
      dtype='object')

In [ ]:
GKV_data.columns

Index(['GKV', 'gdp_growth', 'volume', '('GARCH', 'normal', 0)',
       '('GARCH', 'normal', 1)', '('GARCH', 'gaussian', 0)',
       '('GARCH', 'gaussian', 1)', '('GARCH', 'ged', 0)',
       '('GARCH', 'ged', 1)', '('FIGARCH', 'normal', 0)',
       '('FIGARCH', 'normal', 1)', '('FIGARCH', 'gaussian', 0)',
       '('FIGARCH', 'gaussian', 1)', 'oil', 'gold', 'nasdaq', 'djia', 'vix',
       'interest_rate', 'inflation_rate', 'unemployment_rate', 'usd_eur',
       'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn', 'gt_data',
       'log_returns', '('APARCH', 'studentst', 1)'],
      dtype='object')

In [ ]:
GKYZV_data.columns

Index(['GKYZV', '('FIGARCH', 'normal', 0)', '('FIGARCH', 'normal', 1)',
       '('FIGARCH', 'gaussian', 0)', '('FIGARCH', 'gaussian', 1)',
       'gdp_growth', 'volume', '('GARCH', 'normal', 0)',
       '('GARCH', 'normal', 1)', '('GARCH', 'gaussian', 0)',
       '('GARCH', 'gaussian', 1)', '('GARCH', 'ged', 0)',
       '('GARCH', 'ged', 1)', 'oil', 'gold', 'nasdaq', 'djia', 'vix',
       'interest_rate', 'inflation_rate', 'unemployment_rate', 'usd_eur',
       'usd_jpy', 'usd_gbp', 'usd_cny', 'usd_cad', 'usd_mxn', 'gt_data',
       'log_returns', '('APARCH', 'studentst', 1)'],
      dtype='object')

In [ ]:
filename = 'HV_data.csv'
HV_data.to_csv(path_1 + filename)

In [ ]:
filename = 'CCV_data.csv'
CCV_data.to_csv(path_1 + filename)

In [ ]:
filename = 'PV_data.csv'
PV_data.to_csv(path_1 + filename)

In [ ]:
filename = 'GKV_data.csv'
GKV_data.to_csv(path_1 + filename)

In [ ]:
filename = 'RSV_data.csv'
RSV_data.to_csv(path_1 + filename)

In [ ]:
filename = 'YZV_data.csv'
YZV_data.to_csv(path_1 + filename)

In [ ]:
filename = 'GKYZV_data.csv'
GKYZV_data.to_csv(path_1 + filename)